In [1]:
import pandas as pd
import re
from tqdm import tqdm
import json

In [7]:
!wget -O /content/corpus1.json https://raw.githubusercontent.com/eanor/nlp_project_2024/main/corpus/corpus19.json
!wget -O /content/corpus2.json https://raw.githubusercontent.com/eanor/nlp_project_2024/main/corpus/corpus20.json
!wget -O /content/corpus3.json https://raw.githubusercontent.com/eanor/nlp_project_2024/main/corpus/corpus21.json

--2024-03-24 20:15:31--  https://raw.githubusercontent.com/eanor/nlp_project_2024/main/corpus/corpus19.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5517518 (5.3M) [text/plain]
Saving to: ‘/content/corpus1.json’

/content/corpus1.js 100%[===================>]   5.26M  --.-KB/s    in 0.08s   

2024-03-24 20:15:31 (64.6 MB/s) - ‘/content/corpus1.json’ saved [5517518/5517518]

--2024-03-24 20:15:31--  https://raw.githubusercontent.com/eanor/nlp_project_2024/main/corpus/corpus20.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [8]:
def get_corpus(path):
    with open(path,'r', encoding='utf-8') as f:
        corpus = json.load(f)
    return corpus

In [9]:
def reduce_cont(d):
  '''Для ELMO предложений оказалось слишком много, ограничили 15-ю, иначе будет очень долго работать'''

  for k in d.keys():
    if len(d[k]) > 15:
      d[k] = d[k][:15]

  return d

In [10]:
corpus1 = reduce_cont(get_corpus('/content/corpus1.json'))
corpus2 = reduce_cont(get_corpus('/content/corpus2.json'))
corpus3 = reduce_cont(get_corpus('/content/corpus3.json'))

In [11]:
!wget -O /content/annotated_testset.tsv https://raw.githubusercontent.com/eanor/nlp_project_2024/main/models/chat_targets.tsv

# первые 20 целевых слов, те же, что и для GigaChat

--2024-03-24 20:15:41--  https://raw.githubusercontent.com/eanor/nlp_project_2024/main/models/chat_targets.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1294 (1.3K) [text/plain]
Saving to: ‘/content/annotated_testset.tsv’

/content/annotated_ 100%[===================>]   1.26K  --.-KB/s    in 0s      

2024-03-24 20:15:41 (129 MB/s) - ‘/content/annotated_testset.tsv’ saved [1294/1294]



In [12]:
df = pd.read_csv('/content/annotated_testset.tsv', sep="\t", header=None)

In [13]:
target_words=[]
cnt = 1
for w in df[0]:
    if cnt == 21: # возьмем первые 20 слов, вместо 100
      break
    target_words.append(w)
    cnt += 1

In [14]:
! pip install --upgrade simple_elmo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.2 MB/s eta 0:00:00


In [15]:
from simple_elmo import ElmoModel

model = ElmoModel()

In [16]:
!wget http://vectors.nlpl.eu/repository/20/196.zip

--2024-03-24 20:16:17--  http://vectors.nlpl.eu/repository/20/196.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206986351 (197M) [application/zip]
Saving to: ‘196.zip’

196.zip             100%[===================>] 197.40M  24.9MB/s    in 8.8s    

2024-03-24 20:16:26 (22.5 MB/s) - ‘196.zip’ saved [206986351/206986351]



In [17]:
import tensorflow as tf

model.load("/content/196.zip")

/usr/local/lib/python3.10/dist-packages/simple_elmo/model.py:531: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  lstm_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(


'The model is now loaded.'

In [18]:
import numpy as np

In [14]:
!mkdir /content/experiment_dir/

In [19]:
import os

newdir = '/content/experiment_dir/word_representations/'

rep_dir_c1 = newdir + "c1/"
rep_dir_c2 = newdir + "c2/"
rep_dir_c3 = newdir + "c3/"

os.makedirs(rep_dir_c1, exist_ok=True)
os.makedirs(rep_dir_c2, exist_ok=True)
os.makedirs(rep_dir_c3, exist_ok=True)

In [32]:
def elmocounter(rep_dir, target_words, corpus):
    allresult = []
    for word in tqdm(target_words):
        vectors = []
        print('\n-----', word.upper(), '-----')
        for sent in tqdm(corpus[word]):
            if word in sent:
                word_idx = sent.index(word)
                word_vector = model.get_elmo_vectors([sent])[0][word_idx]
                vectors.append(word_vector)
        vectors = np.array(vectors)
        np.save(rep_dir + word + ".npy", vectors)
        avg = np.mean(vectors, axis=0)
        allresult.append({word: avg})
    return allresult

In [58]:
print("Extracting representations for CORPUS19...")
c1_reps = elmocounter('/content/experiment_dir/word_representations/c1/', target_words, corpus1)

Extracting representations for CORPUS19...


  0%|          | 0/1 [00:00<?, ?it/s]


----- АВТОРИТЕТ -----



100%|██████████| 1/1 [00:48<00:00, 48.31s/it]


In [59]:
print("Extracting representations for CORPUS20...")
c2_reps = elmocounter('/content/experiment_dir/word_representations/c2/', target_words, corpus2)

Extracting representations for CORPUS20...


  0%|          | 0/1 [00:00<?, ?it/s]


----- АВТОРИТЕТ -----



100%|██████████| 1/1 [00:28<00:00, 28.76s/it]


In [36]:
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

In [37]:
from google.colab import files

def download_reps(c):
    czip = c + '.zip'
    zipf = zipfile.ZipFile(czip, 'w', zipfile.ZIP_DEFLATED)
    zipdir('/content/experiment_dir/word_representations/' + c + '/', zipf)
    zipf.close()

    files.download("/content/" + czip)

In [60]:
print("Extracting representations for CORPUS21...")
c3_reps = elmocounter('/content/experiment_dir/word_representations/c3/', target_words, corpus3)

Extracting representations for CORPUS21...


  0%|          | 0/1 [00:00<?, ?it/s]


----- АВТОРИТЕТ -----



100%|██████████| 1/1 [00:36<00:00, 36.00s/it]


In [65]:
download_reps('c1')
download_reps('c2')
download_reps('c3')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
from scipy.spatial import distance

def get_cosine_dist(Xs, Ys):

    return distance.cosine(Xs, Ys)

In [62]:
dists_1_2 = []
dists_2_3 = []
dists_1_3 = []

for id, target in tqdm(enumerate(target_words)):

    c1_reps_d = c1_reps[id]
    c2_reps_d = c2_reps[id]
    c3_reps_d = c3_reps[id]

    dist_1_2 = get_cosine_dist(c1_reps_d[target], c2_reps_d[target])
    dist_2_3 = get_cosine_dist(c2_reps_d[target], c3_reps_d[target])
    dist_1_3 = get_cosine_dist(c1_reps_d[target], c3_reps_d[target])

    dists_1_2.append(dist_1_2)
    dists_2_3.append(dist_2_3)
    dists_1_3.append(dist_1_3)

pd.DataFrame({"word": target_words, "change_1_2": dists_1_2, "change_2_3": dists_2_3, "change_1_3": dists_1_3}).to_csv("prediction_elmo.tsv", sep="\t", index=False, header=False)

1it [00:00, 848.19it/s]
